In [1]:
import torch
import functions
import modules
import math
from optimizer import SGD

torch.set_grad_enabled(False)

## Test.py

## Generate data

In [2]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [3]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [4]:
train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)
train_target = convert_to_one_hot_labels(train_input, train_target)
test_target = convert_to_one_hot_labels(test_input, test_target)

In [5]:
train_input = 0.9 * train_input

# TODO Normalization of the data

## Mini-Project Architecture

In [6]:
model1 = modules.Sequential(modules.Linear(2, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 2),
                                  modules.MSELoss()
                                 )

model2 = modules.Sequential(modules.Linear(2, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 25),
                                  modules.ReLU(),
                                  modules.Linear(25, 2),
                                  modules.MSELoss()
                                 )

In [7]:
nb_epochs = 50
lr = 1e-5
mini_batch_size = 100
optimizer1 = SGD(model1.param(), lr=lr)
optimizer2 = SGD(model2.param(), lr=lr)

In [8]:
for e in range(nb_epochs):
    sum_loss = 0
    for b in range(0, train_input.size(0), mini_batch_size):
        output, loss = model1.forward(train_input.narrow(0, b, mini_batch_size), train_target.narrow(0, b, mini_batch_size))
        optimizer1.zero_grad()
        grad = model1.backward()
        sum_loss = sum_loss + loss.item()
        optimizer1.step()
    print("Iteration {0:}: loss = {1:.3f}".format(e+1, sum_loss), end='\r', flush=True)
loss_train = sum_loss

In [9]:
output_test, loss_test = model1.forward(test_input, test_target)

In [10]:
print("Model 1 with TanH")
print("Training loss: {:.3f}".format(loss_train))
print("Test loss: {:.3f}".format(loss_test))

Model 1 with TanH
Training loss: 1.462
Test loss: 0.126


In [11]:
for e in range(nb_epochs):
    sum_loss = 0
    for b in range(0, train_input.size(0), mini_batch_size):
        output, loss = model2.forward(train_input.narrow(0, b, mini_batch_size), train_target.narrow(0, b, mini_batch_size))
        optimizer2.zero_grad()
        grad = model2.backward()
        sum_loss = sum_loss + loss.item()
        optimizer2.step()
    print("Iteration {0:}: loss = {1:.3f}".format(e+1, sum_loss), end='\r', flush=True)
loss_train = sum_loss

In [12]:
output_test, loss_test = model2.forward(test_input, test_target)

In [13]:
import torch
torch.empty(5)

tensor([ 0.0000e+00, -4.6566e-10, -8.7476e-29, -2.5250e-29,         nan])

In [14]:
print("Model 2 with ReLu")
print("Training loss: {:.3f}".format(loss_train))
print("Test loss: {:.3f}".format(loss_test))

Model 2 with ReLu
Training loss: 1.966
Test loss: 0.168
